In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('../input/santander-value-prediction-challenge/train.csv')
test_df = pd.read_csv('../input/santander-value-prediction-challenge/test.csv', nrows=100)

In [ ]:
print("All Features in Train data with NaN Values =", str(train_df.columns[train_df.isnull().sum() != 0].size) )
# print("All Features in Test data with NaN Values =", str(test_df.columns[train_df.isnull().sum() != 0].size) )

## Remove constant columns from data

In [ ]:
const_columns_to_remove = []
for col in train_df.columns:
    if col != 'ID' and col != 'target':
        if train_df[col].std() == 0:
            const_columns_to_remove.append(col)

# Now remove that array of const columns from the data
train_df.drop(const_columns_to_remove, axis=1, inplace=True)
test_df.drop(const_columns_to_remove, axis=1, inplace=True)

## Remove Duplicate Columns

In [ ]:
train_df = train_df.loc[:,~train_df.columns.duplicated()]

### handle sparse data

In [ ]:
def drop_parse_from_df(df):
    column_list_to_drop_data_from = [i for i in df.columns if not i in ['ID', 'target'] ]
    for column in column_list_to_drop_data_from:
        if len(np.unique(df[column])) < 2:
            df.drop(column, axis=1, inplace=True)
            df.drop(column, axis=1, inplace=True)
    return df


train_df = drop_parse_from_df(train_df)
print('Rows and Columns in train_df after removing sparse ', format(train_df.shape))


### Split data into Train and Test for Model Training

In [ ]:
X_train = train_df.drop(['ID', 'target'], axis=1)

y_train = np.log1p(train_df['target'].values)

X_test = test_df.drop('ID', axis=1)
X_train_split, X_validation, y_train_split, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
## LightGBM

In [ ]:
def run_light_gbm(train_X, train_y, validation_X, validation_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }